In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt


In [ ]:
# Step 1: Define the CNN model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # Adjust input size based on your image dimensions
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)  # Flatten the tensor
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Step 2: Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
# Step 3: Define class weights (adjust based on your dataset)
# Example: If class 0 has fewer samples than class 1, assign a higher weight to class 0
class_weights = torch.tensor([2.0, 1.0])  # [weight for class 0, weight for class 1]
class_weights = class_weights.to(device)

In [ ]:
# Step 4: Define the loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
# Step 5: Define the optimizer
model = CNN(num_classes=2).to(device)  # 2 classes: 0.NoError and 1.Error
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Step 6: Load the dataset
# Define the path to your dataset
data_dir = "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/TrainingData"

# Define transformations for the training data
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize images to 32x32
    transforms.ToTensor(),         # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load the dataset using ImageFolder
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Create a DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# Step 7: Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # Move data to the appropriate device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

print("Training complete!")

In [ ]:
# Save the trained model
torch.save(model.state_dict(), "trained_CNNmodel.pth")
print("Model saved.")

##Analysis

In [ ]:
# Step 1: Define a test dataset
class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.image_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith(".jpg")]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, image_path  # Return image path for visualization

In [ ]:
# Step 2: Load test data
test_dir = "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Processed_Frames/X.Test/Images"
test_dataset = TestDataset(test_dir=test_dir, transform=transform)

# Check if test dataset is empty
if len(test_dataset) == 0:
    print("Error: No .jpg files found in the test directory.")
else:
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
    # Step 3: Evaluate the model on the test set
    model.eval()
    test_predictions = []
    test_image_paths = []

    with torch.no_grad():
        for images, paths in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_predictions.extend(predicted.cpu().numpy())
            test_image_paths.extend(paths)

In [ ]:
    # Step 4: Analyze predictions
    for image_path, prediction in zip(test_image_paths, test_predictions):
        print(f"Image: {image_path}, Predicted: {'Error' if prediction == 1 else 'NoError'}")

In [ ]:
    # Step 5: Visualize a few test predictions
    num_samples = min(5, len(test_image_paths))  # Ensure we don't exceed the number of available samples
    for i in range(num_samples):
        image_path = test_image_paths[i]
        prediction = test_predictions[i]

        image = Image.open(image_path).convert("RGB")
        plt.imshow(image)
        plt.title(f"Predicted: {'Error' if prediction == 1 else 'NoError'}")
        plt.axis("off")
        plt.show()

##Applying GradCam

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import numpy as np
import cv2

# Step 1: Define the CNN model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)

        # Adjust the input size for fc1 to match the saved model
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # Input size: 32 * 8 * 8 = 2048
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor while preserving the batch dimension
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Step 2: Load the trained model
model = CNN(num_classes=2)  # 2 classes: 0.NoError and 1.Error
model.load_state_dict(torch.load("trained_CNNmodel.pth", weights_only=True))  # Load your trained model
model.eval()  # Set the model to evaluation mode

# Step 3: Define the Grad-CAM function
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None

        # Hook into the target layer
        target_layer.register_forward_hook(self.save_activations)
        target_layer.register_backward_hook(self.save_gradients)

    def save_activations(self, module, input, output):
        self.activations = output

    def save_gradients(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def forward(self, x):
        return self.model(x)

    def backward(self, outputs):
        self.model.zero_grad()
        outputs.backward(torch.ones_like(outputs))

    def generate(self, x, class_idx=None):
        # Forward pass
        outputs = self.forward(x)
        print(f"Model outputs shape: {outputs.shape}")  # Debug statement
        if class_idx is None:
            class_idx = torch.argmax(outputs, dim=1).item()

        # Backward pass
        self.backward(outputs[:, class_idx].sum())

        # Compute Grad-CAM
        weights = torch.mean(self.gradients, dim=[2, 3], keepdim=True)
        cam = torch.sum(weights * self.activations, dim=1, keepdim=True)
        cam = F.relu(cam)  # Apply ReLU to highlight positive influences
        cam = F.interpolate(cam, size=(x.shape[2], x.shape[3]), mode="bilinear", align_corners=False)
        cam = cam - cam.min()
        cam = cam / cam.max()
        return cam.squeeze().cpu().detach().numpy()

# Step 4: Load and preprocess an example image
def preprocess_image(image_path, target_size=(32, 32)):
    transform = transforms.Compose([
        transforms.Resize(target_size),  # Resize to match model input size
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    image = datasets.folder.default_loader(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Step 5: Apply Grad-CAM to an image
def apply_gradcam(image_path, model, target_layer, target_size=(32, 32)):
    # Preprocess the image
    image = preprocess_image(image_path, target_size)
    image = image.to(device)

    # Initialize Grad-CAM
    grad_cam = GradCAM(model, target_layer)

    # Generate the heatmap
    heatmap = grad_cam.generate(image)

    # Load the original image for visualization
    original_image = cv2.imread(image_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # Resize the heatmap to match the original image resolution
    heatmap = cv2.resize(heatmap, (original_image.shape[1], original_image.shape[0]))

    # Normalize the heatmap
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Overlay the heatmap on the original image
    superimposed_img = heatmap * 0.4 + original_image * 0.6
    superimposed_img = np.uint8(superimposed_img)

    # Display the results
    plt.figure(figsize=(15, 10))
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_image)
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title("Grad-CAM Heatmap")
    plt.imshow(superimposed_img)
    plt.axis("off")

    plt.show()

# Step 6: Define the target layer for Grad-CAM
# Choose the last convolutional layer in your model
target_layer = model.conv2

# Step 7: Apply Grad-CAM to an example image
image_path = "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Processed_Frames/With Background/GoodFormWithBackground/Tr1_NIE/frame_0300.jpg"  # Replace with the path to your high-resolution image
apply_gradcam(image_path, model, target_layer, target_size=(32, 32))  # Use the correct input size (e.g., 32x32)